In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_parquet("../data/train.parquet")
train_df

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
216716091,12899776,1737908,1661723987,0
216716092,12899777,384045,1661723976,0
216716093,12899777,384045,1661723986,0
216716094,12899778,561560,1661723983,0


In [4]:
seven_days = 7*24*60*60
train_cutoff = train_df.ts.max() - seven_days
train_df = train_df[train_df.ts <= train_cutoff]

In [7]:
train_df.session.max()

11098527

In [8]:
clicks_count = train_df[train_df['type'] == 0].groupby("aid")[['aid']].agg("size")
carts_count = train_df[train_df['type'] == 1].groupby("aid")[['aid']].agg("size")
orders_count = train_df[train_df['type'] == 2].groupby("aid")[['aid']].agg("size")
train_df['item_click_count'] = train_df['aid'].map(clicks_count)
train_df['item_carts_count'] = train_df['aid'].map(carts_count)
train_df['item_orders_count'] = train_df['aid'].map(orders_count)
train_df = train_df.fillna(0)

/tmp/ipykernel_17050/87033043.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['item_click_count'] = train_df['aid'].map(clicks_count)
/tmp/ipykernel_17050/87033043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['item_carts_count'] = train_df['aid'].map(carts_count)
/tmp/ipykernel_17050/87033043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [9]:
train_df

,session,aid,ts,type,item_click_count,item_carts_count,item_orders_count
0,0,1517085,1659304800,0,79.0,12.0,3.0
1,0,1563459,1659304904,0,64.0,0.0,0.0
2,0,1309446,1659367439,0,4115.0,493.0,87.0
3,0,16246,1659367719,0,1033.0,117.0,38.0
4,0,1781822,1659367871,0,39.0,3.0,1.0
...,...,...,...,...,...,...,...
201331461,11098523,175715,1661119197,0,1914.0,10.0,0.0
201331464,11098524,1088524,1661119198,0,5474.0,422.0,180.0
201331469,11098525,182927,1661119199,0,1324.0,124.0,39.0
201331485,11098526,510055,1661119199,0,1205.0,173.0,75.0


In [10]:
item_feat = train_df.drop_duplicates("aid")[['aid','item_click_count','item_carts_count','item_orders_count']]
item_feat

,aid,item_click_count,item_carts_count,item_orders_count
0,1517085,79.0,12.0,3.0
1,1563459,64.0,0.0,0.0
2,1309446,4115.0,493.0,87.0
3,16246,1033.0,117.0,38.0
4,1781822,39.0,3.0,1.0
...,...,...,...,...
201297273,385645,1.0,0.0,0.0
201304587,511976,1.0,0.0,0.0
201308463,720469,2.0,0.0,0.0
201324592,582525,1.0,0.0,0.0


In [11]:
item_feat.to_parquet("../data/feat/item_feat.parquet")